# [[Pandas] 범주형 변수를 더미 변수(dummy  variable)로 바꾸기 ](https://zephyrus1111.tistory.com/91)

In [1]:
import pandas as pd
import numpy as np

In [6]:
df = pd.DataFrame({'gender':['male','female','male','male','female'],
                     'height':[175, 165, 180, 169, 188],
                     'nation': ['USA', 'Korea', 'China', 'Korea', 'Brazil'],
                    'married':[0,1,1,1,0]})

In [8]:
df

,gender,height,nation,married
0,male,175,USA,0
1,female,165,Korea,1
2,male,180,China,1
3,male,169,Korea,1
4,female,188,Brazil,0


In [9]:
pd.get_dummies(df)

,height,married,gender_female,gender_male,nation_Brazil,nation_China,nation_Korea,nation_USA
0,175,0,0,1,0,0,0,1
1,165,1,1,0,0,0,1,0
2,180,1,0,1,0,1,0,0
3,169,1,0,1,0,0,1,0
4,188,0,1,0,1,0,0,0


아래 코드를 이용해서 더미변수로 처리할 컬럼을 지정할 수 있다.

In [12]:
pd.get_dummies(df, columns=['gender', 'nation', 'married'], drop_first=True)

,height,gender_male,nation_China,nation_Korea,nation_USA,married_1
0,175,1,0,0,1,0
1,165,0,0,1,0,1
2,180,1,1,0,0,1
3,169,1,0,1,0,1
4,188,0,0,0,0,0


drop_first 옵션을 이용해서 컬럼에 속한 한 개의 범주를 제거한다.  
제외되는 가변수는 숫자인 경우는 가장 작은 수, 문자열이면 알파벳순으로 가장 앞에 있는 문자열이다.

예를 들어 gender는 male, female 값이 있는데 더미변수로 변환하면 gender_male 칼럼만 생성된다. 여기서 1은 male, 0은 female을 의미한다.

하지만 get_dummies 함수는 원하는 범주를 제외시켜주는 기능이 없다. 따라서 우리가 원하는 범주를 제외하기 위해서는 함수를 따로 만들어야 한다. 아래 코드를 이용할 수 있다.

In [13]:
def get_dummy(df,columns,base_value=None):
    '''
    df : 데이터프레임
    columns : 가변수로 변환할 칼럼들
    base_value : {가변수로 변환할 칼럼명 : 제외시킬 범주}
    '''
    for c in columns:
        num_level = len(set(df[c])) ## 유니크한 원소의 개수
        uniq_element = sorted(list(set(df[c])))[1:] ## 정렬했을 때 첫 번째 범주를 제외
        
        if base_value: ## 제외시킬 범주가 있을 경우 해당 범주를 제외
            if c in base_value.keys():
                assert base_value[c] in list(set(df[c])), f'{base_value[c]} is not contained in {c}'
                uniq_element = sorted(list(set(df[c])-{base_value[c]}))
                    
        data = dict()
        for i in range(num_level-1):
            dummy_data = []
            val = uniq_element[i]
            for d in df[c]: ## 해당 범주인 경우만 1 나머지는 0
                if d == val:
                    dummy_data.append(1)
                else:
                    dummy_data.append(0)
            col_name = c+'_'+str(val) ## 갸변수의 칼럼명 지정
            data[col_name] = dummy_data
        
        temp_df = pd.DataFrame(data)
        df = pd.concat([df,temp_df],axis=1) ## 가변수 칼럼을 원 데이터 뒤에 결합
 
    return df.drop(columns=columns) ## 원 범주 칼럼은 제외

아래 코드는 위 함수를 이용하여 married 칼럼에는 0, nation 칼럼에는 Korea를 제외시키고 가변수로 변환한다.

In [14]:
get_dummy(df,['married','nation','gender'],{'married':1,'nation':'Korea'})

,height,married_0,nation_Brazil,nation_China,nation_USA,gender_male
0,175,1,0,0,1,1
1,165,0,0,0,0,0
2,180,0,0,1,0,1
3,169,0,0,0,0,1
4,188,1,1,0,0,0


결과를 보면 원하는 범주가 제외되고 더미변수로 바뀌었다.